This program will train the data using different supervised learning methods.Run this program after Clustering_ThirdStep.ipynb

In [0]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import time as t
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
dataset = pd.read_feather("/content/drive/My Drive/data_features")
test = pd.read_feather('/content/gdrive/My Drive/Big data/Data files/whatever the hell.feather')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# reclassify hours into categories
def preprocess_hour(val):
    if val>=17:
        return 1 #evening
    elif val>7 and val<17:
        return 2 #mid day
    elif val==0:
        return 3 #midnight
    else:
        return 4 #early morning
dataset['hour']=dataset['hour'].apply(preprocess_hour)

# create additional features as dummy variables for weekdays
dataset = pd.get_dummies(dataset, columns=['weekday'])
dataset.rename(columns={'weekday_0':'mon','weekday_1':'tues','weekday_2':'wed','weekday_3':'thu','weekday_4':'fri','weekday_5':'sat','weekday_6':'sun'},inplace=True)

In [0]:
# set variable types
dataset[['pickup_longitude', 'pickup_latitude', 'dropoff_latitude', 'dropoff_longitude', 'haversine_distance', 'fare_amount']] = dataset[['pickup_longitude', 'pickup_latitude', 'dropoff_latitude', 'dropoff_longitude', 'haversine_distance', 'fare_amount']].astype('float32')
dataset[['passenger_count', 'month', 'hour', 'year']] = dataset[['passenger_count', 'month', 'hour', 'year']].astype('int16')

dataset.head(5)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,month,hour,date,year,haversine_distance,mon,tues,wed,thu,fri,sat,sun
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21+00:00,-73.844315,40.721317,-73.841614,40.712276,1,6,1,2009-06-15,2009,0.640668,1,0,0,0,0,0,0
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16+00:00,-74.016045,40.711304,-73.979271,40.782005,1,1,2,2010-01-05,2010,5.252153,0,1,0,0,0,0,0
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00+00:00,-73.982735,40.761269,-73.991241,40.750561,2,8,3,2011-08-18,2011,0.863655,0,0,0,1,0,0,0
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42+00:00,-73.987129,40.733143,-73.991570,40.758091,1,4,4,2012-04-21,2012,1.739877,0,0,0,0,0,1,0
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00+00:00,-73.968094,40.768009,-73.956657,40.783764,1,3,4,2010-03-09,2010,1.242569,0,1,0,0,0,0,0


In [0]:
# NEW CLUSTERING
z=t.time()

sample_data = dataset.sample(frac = 0.6)
from sklearn.cluster import MiniBatchKMeans
cost = [] 
for i in range(9, 16): 
    KM_pickup = MiniBatchKMeans(n_clusters = i) 
    KM_pickup.fit(sample_data[['pickup_latitude', 'pickup_longitude']])
    cost.append(KM_pickup.inertia_)

from sklearn.cluster import MiniBatchKMeans
cost = [] 
for i in range(9, 16): 
    KM_dropoff = MiniBatchKMeans(n_clusters = i) 
    KM_dropoff.fit(sample_data[['dropoff_latitude', 'dropoff_longitude']])
    cost.append(KM_dropoff.inertia_)

from sklearn.cluster import MiniBatchKMeans
KM_pickup = MiniBatchKMeans(n_clusters = 10) 
KM_pickup_train = KM_pickup.fit(dataset[['pickup_latitude', 'pickup_longitude']])

KM_pickup_test = KM_pickup.fit_predict(test[['pickup_latitude', 'pickup_longitude']])

dataset['pickup_cluster'] = KM_pickup_train.labels_
pickup_labels = KM_pickup_train.labels_

from sklearn.cluster import MiniBatchKMeans
KM_dropoff = MiniBatchKMeans(n_clusters = 13)
KM_dropoff_train = KM_dropoff.fit(dataset[['dropoff_latitude', 'dropoff_longitude']])

KM_pickup_test = KM_pickup.fit_predict(test[['dropoff_latitude', 'dropoff_longitude']])

dataset['dropoff_cluster'] = KM_dropoff.fit(dataset[['dropoff_latitude', 'dropoff_longitude']]).labels_
dropoff_labels = KM_dropoff_train.labels_

print("Time:", t.time()-z)
dataset.head(5)

In [0]:
# OLD CLUSTERING
z=t.time()
from sklearn.cluster import MiniBatchKMeans

KM_pickup = MiniBatchKMeans(n_clusters = 10) 
KM_pickup_train = KM_pickup.fit(dataset[['pickup_latitude', 'pickup_longitude']])

dataset['pickup_cluster'] = KM_pickup_train.labels_
pickup_labels = KM_pickup_train.labels_

KM_dropoff = MiniBatchKMeans(n_clusters = 13)
KM_dropoff_train = KM_dropoff.fit(dataset[['dropoff_latitude', 'dropoff_longitude']])

dataset['dropoff_cluster'] = KM_dropoff.fit(dataset[['dropoff_latitude', 'dropoff_longitude']]).labels_
dropoff_labels = KM_pickup_train.labels_

#dataset.to_feather('dataset.feather')
print("Time:", t.time()-z)

dataset.head(5)

Time: 603.4898126125336


,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,month,hour,date,year,haversine_distance,mon,tues,wed,thu,fri,sat,sun,pickup_cluster,dropoff_cluster
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21+00:00,-73.844315,40.721317,-73.841614,40.712276,1,6,1,2009-06-15,2009,0.640668,1,0,0,0,0,0,0,2,8
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16+00:00,-74.016045,40.711304,-73.979271,40.782005,1,1,2,2010-01-05,2010,5.252153,0,1,0,0,0,0,0,8,7
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00+00:00,-73.982735,40.761269,-73.991241,40.750561,2,8,3,2011-08-18,2011,0.863655,0,0,0,1,0,0,0,3,0
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42+00:00,-73.987129,40.733143,-73.991570,40.758091,1,4,4,2012-04-21,2012,1.739877,0,0,0,0,0,1,0,0,0
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00+00:00,-73.968094,40.768009,-73.956657,40.783764,1,3,4,2010-03-09,2010,1.242569,0,1,0,0,0,0,0,3,3


In [0]:
# divide dataset into "attributes" and "labels"
x = dataset[['passenger_count', 'month','mon','tues','wed','thu','fri','sat','sun','hour', 'year', 'pickup_cluster','dropoff_cluster','haversine_distance']]
y = dataset['fare_amount'].values

In [0]:
sampleData = dataset.sample(frac = 0.2, random_state = 42)

# divide dataset into "attributes" and "labels"
xSamp = sampleData[['passenger_count', 'month','mon','tues','wed','thu','fri','sat','sun','hour', 'year', 'pickup_cluster','dropoff_cluster','haversine_distance']]
ySamp = sampleData['fare_amount'].values

sampleData.shape

(10683504, 22)

In [0]:
# LINEAR REGRESSION
z = t.time()

# split 70% into training and 30% into testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

# train model
model = LinearRegression(n_jobs=-1)
model.fit(x_train, y_train)

# predict values
y_pred = model.predict(x_test)

# evauluate linear regression model
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print("Time: ", t.time()-z)

Mean Absolute Error: 2.2975100510005473
Mean Squared Error: 19.73922589877897
Root Mean Squared Error: 4.44288486220147
Time:  48.821569204330444


In [0]:
# SAMPLED LINEAR REGRESSION
z = t.time()

# split 70% into training and 30% into testing
x_train, x_test, y_train, y_test = train_test_split(xSamp, ySamp, test_size=0.3, random_state=42)

# train model
model = LinearRegression()
model.fit(x_train, y_train)

# predict values
y_pred = model.predict(x_test)

# evauluate linear regression model  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print("Time: ", t.time()-z)

Root Mean Squared Error: 4.3322554271571585
Time:  7.298843622207642


In [0]:
from sklearn.metrics import r2_score
print("R^2 Score:", r2_score(y_test, y_pred))

In [0]:
# GRADIENT BOOSTING REGRESSION
z=t.time()

from sklearn.ensemble import GradientBoostingRegressor

# split 80% into training and 20% into testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

model = GradientBoostingRegressor(n_estimators = 50, learning_rate = 0.25, max_depth=3, random_state=42, warm_start = True)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print("Time:", t.time()-z)

Root Mean Squared Error: 4.035744339796134
Time: 4862.593821287155


In [0]:
# SAMPLED GRADIENT BOOSTING REGRESSION
z=t.time()

from sklearn.ensemble import GradientBoostingRegressor

x_train, x_test, y_train, y_test = train_test_split(xSamp, ySamp, test_size=0.2, random_state=42)

model = GradientBoostingRegressor(n_estimators = 50, learning_rate = 0.25, max_depth=3, random_state=42, warm_start = True)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print("Time:", t.time()-z)

Root Mean Squared Error: 3.8916012350049707
Time: 981.917905330658


In [0]:
# GRADIENT BOOSTING REGRESSION
z=t.time()

from sklearn.ensemble import GradientBoostingRegressor

# split 70% into training and 30% into testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

model = GradientBoostingRegressor(n_estimators = 100, learning_rate = 0.25, max_depth=3, 
                                  random_state=42, warm_start = True)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print("Time:", t.time()-z)

Root Mean Squared Error: 3.9913984263796065
Time: 8010.957782030106


In [0]:
# GRADIENT BOOSTING REGRESSION
z=t.time()

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingRegressor

# split 70% into training and 30% into testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

model = GradientBoostingRegressor(n_estimators = 100, learning_rate = 0.20, max_depth=3, 
                                  random_state=42, warm_start = True)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print("Time:", t.time()-z)

Root Mean Squared Error: 4.052837399708199
Time: 8066.647850036621


In [0]:
# GRADIENT BOOSTING REGRESSION
z=t.time()

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingRegressor

# split 70% into training and 30% into testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

model = GradientBoostingRegressor(n_estimators = 200, learning_rate = 0.25, max_depth=3, 
                                  random_state=42, warm_start = True)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print("Time:", t.time()-z)

Root Mean Squared Error: 3.98464763854101
Time: 15194.329134225845


In [0]:
# RANDOM FORREST REGRESSION
z = t.time()

# split 90% into training and 10% into testing, with random_state 1 for reproducible results
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=1)

# Train and instantiate model with 10 decision trees
model = RandomForestRegressor(max_depth = 4, n_estimators = 100, random_state = 1, oob_score = True, n_jobs = -1)
model.fit(x_train, y_train)

# predict and compute mean absolute error
y_pred = model.predict(x_test)

print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print("Time: ", t.time()-z)

Root Mean Squared Error: 4.328028102171624
Time:  5517.89039850235


In [0]:
# RANDOM FORREST REGRESSION
z = t.time()

# split 90% into training and 10% into testing, with random_state 1 for reproducible results
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=1)

# Train and instantiate model with 10 decision trees
model = RandomForestRegressor(max_depth = 3, n_estimators = 100, bootstrap=True, oob_score = True, n_jobs = -1, random_state = 1)
model.fit(x_train, y_train)

# predict and compute mean absolute error
y_pred = model.predict(x_test)

print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

print("Time: ", t.time()-z)

Root Mean Squared Error: 4.327835202869214
Time:  6405.718513965607
